In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np

In [ ]:
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [ ]:
#u at t=0
u_0= np.zeros(41,int) 

#initial condition
for i in range(0,41): 
    if i<=10:
        u_0[i] = 1
    else:
        u_0[i] = 2

A = np.zeros([41,41],int)
A = np.matrix(A)
#A matrix
for i in range(0,41):
    A[i,i:i+3] += 1

#u at t=1
u_1 = NumPyLSsolver(A, u_0).run()
print("Classical Solution:\t", np.round(u_1['solution'], 5))


In [ ]:
# u at t=2
u_2 = NumPyLSsolver(A, u_1['solution']).run()
print("Classical Solution:\t", np.round(u_2['solution'], 5))

In [ ]:

orig_size = len(u_0)
A, u_0, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(A, u_0)

# Initialize eigenvalue finding module
eigs = create_eigs(A, 3, 50, False)
num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=u_01)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algor = HHL(A, u_0, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

In [ ]:
#u at t=1 using HHL Algorithm
u_1_q = algor.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:\t\t", (np.round(u_1_q['solution'],5)))